In [65]:
#Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#sklearn 
from sklearn.linear_model import LogisticRegression
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.model_selection import cross_validate, GridSearchCV, train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, log_loss, roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.feature_selection import SequentialFeatureSelector, SelectFromModel
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.inspection import permutation_importance
from sklearn.dummy import DummyClassifier

#imblearn to balance training data
from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek
from imblearn.pipeline import make_pipeline as imblearn_make_pipeline
from collections import Counter

# XGB
import xgboost as xgb
from xgboost import XGBClassifier
xgb.set_config(verbosity=0) # to supress warnings

# shap
import shap

#joblib to save trained models
import joblib

# Show pipelines as interactive diagram
from sklearn import set_config; set_config(display='diagram')

In [45]:
# Set random seed for reproducibility reasons
np.random.seed(42)

# Create clean dataframe


## Load dataset

In [46]:
df = pd.read_csv("../../data-cleaned/compas-scores-two-years-clean.csv")

## Filter dataset

ProPublicas reasoning for filter `raw_data`:
>However not all of the rows are useable for the first round of analysis.
>There are a number of reasons remove rows because of missing data:
>* If the charge date of a defendants Compas scored crime was not within 30 days from when the person was arrested, we assume that because of data quality reasons, that we do not have the right offense.
>* We coded the recidivist flag -- `is_recid` -- to be -1 if we could not find a compas case at all.
>* In a similar vein, ordinary traffic offenses -- those with a `c_charge_degree` of 'O' -- will not result in Jail time are removed (only two of them).
>* We filtered the underlying data from Broward county to include only those rows representing people who had either recidivated in two years, or had at least two years outside of a correctional facility.

In [47]:
df = df[
        (df["days_b_screening_arrest"]<=30) 
        & (df["days_b_screening_arrest"]>=-30) 
        & (df["is_recid"]!=-1) 
        & (df["c_charge_degree"]!="O") 
      ].dropna(subset=['score_text'])

In [48]:
query = """WITH peoples AS (
		SELECT 
			p.id, 
			p.name,
			p.sex,
			p.race,
			p.age,
			p.age_cat,
			p.juv_fel_count,
			p.juv_misd_count, 
			p.juv_other_count, 
			p.priors_count,
			p.compas_screening_date, 
			p.days_b_screening_arrest,
			p.c_charge_degree,
			p.c_charge_desc, 
			p.decile_score,
			p.is_recid
		FROM people p),
	compas_data AS (
		SELECT 
			c.marital_status,
			c.rec_supervision_level,
			c.rec_supervision_level_text,
			c.score_text,
			c.person_id 
		FROM compas c 
		WHERE c.type_of_assessment LIKE 'Risk of Recidivism'),
	jail_history AS (
		SELECT 
			person_id, 
			COUNT(person_id) as times_in_jail, 
			SUM(julianday(out_custody) - JULIANDAY(in_custody)) as total_days_in_jail 
		FROM jailhistory p 
		LEFT JOIN people p2 ON p.person_id = p2.id 
		WHERE DATE(p.in_custody) < DATE(p2.compas_screening_date) AND DATE(p2.compas_screening_date) <= '2014-04-01'
		GROUP BY person_id 
		ORDER BY person_id),
	prison_history AS (
		SELECT 
			person_id, 
			COUNT(person_id) as times_in_prison, 
			SUM(julianday(out_custody) - JULIANDAY(in_custody)) as total_days_in_prison 
		FROM prisonhistory p 
		LEFT JOIN people p2 ON p.person_id = p2.id 
		WHERE DATE(p.in_custody) < DATE(p2.compas_screening_date) AND DATE(p2.compas_screening_date) <= '2014-04-01'
		GROUP BY person_id 
		ORDER BY person_id),
	charge_data AS (
		SELECT 
			c.person_id, 
			c.name, 
			COUNT(DISTINCT c.case_number) FILTER(WHERE c.charge_degree NOT LIKE '(0)' OR '(CO3)' OR '(NI0)') AS prior_cases_count,
			SUM(CASE WHEN c.charge_degree LIKE '%(0)%' OR '(CO3)' THEN 1 ELSE 0 END) as total_ordoffens_charges,
			SUM(CASE WHEN c.charge_degree LIKE '%M%' OR '(TC%)' THEN 1 ELSE 0 END) as total_misd_charges,
			SUM(CASE WHEN c.charge_degree LIKE '%F%' THEN 1 ELSE 0 END) as total_felon_charges,
			COALESCE(MIN(c.charge_degree) FILTER(WHERE c.charge_degree LIKE '%M%' OR '(TC%)'),0) as most_sev_misd,
			COALESCE(MAX(c.charge_degree) FILTER(WHERE c.charge_degree LIKE '%F%'),0) as max_sev_fel,
			COALESCE(MIN(c.charge_degree) FILTER(WHERE c.charge_degree LIKE '%F%'),0) as min_sev_fel
		FROM charge c 
		LEFT JOIN people p2 ON c.person_id = p2.id 
		WHERE DATE(c.offense_date) < CASE WHEN DATE(p2.c_offense_date) NOTNULL THEN DATE(p2.c_offense_date) ELSE p2.c_arrest_date END AND DATE(p2.compas_screening_date) <= '2014-04-01' AND charge_degree NOT LIKE 'False Imprisonment' 
		GROUP BY c.person_id 
		ORDER BY c.person_id )
SELECT DISTINCT
	ps.id, 
	ps.name,
	ps.sex,
	ps.race,
	ps.age,
	ps.age_cat,
	ps.juv_fel_count,
	ps.juv_misd_count, 
	ps.juv_other_count, 
	ps.priors_count,
	IFNULL(chd.prior_cases_count,0) AS prior_cases_count,
	IFNULL(chd.total_misd_charges,0) AS total_misd_charges,
	IFNULL(chd.total_felon_charges,0) AS total_felon_charges,
	IFNULL(chd.most_sev_misd,0) AS most_sev_misd,
	IFNULL(chd.max_sev_fel,0) AS max_sev_fel,
	IFNULL(chd.min_sev_fel,0) AS min_sev_fel,
	IFNULL(jh.times_in_jail,0) AS times_in_jail,
	IFNULL(jh.total_days_in_jail,0) AS total_days_in_jail,
	IFNULL(ph.times_in_prison,0) AS times_in_prison,
	IFNULL(ph.total_days_in_prison,0) AS total_days_in_prison,
	IFNULL(chd.total_ordoffens_charges,0) AS total_ordoffens_charges,
	DATE(ps.compas_screening_date) as compas_screening_date, 
	IFNULL(ps.days_b_screening_arrest,0) AS days_b_screening_arrest,
	ps.c_charge_degree AS c_charge_degree_detailed,
	ps.c_charge_desc,
	cd.rec_supervision_level,
	cd.rec_supervision_level_text,
	cd.score_text, 
	ps.decile_score,
	ps.is_recid
FROM peoples ps
LEFT JOIN compas_data cd ON cd.person_id = ps.id 
LEFT JOIN jail_history jh ON jh.person_id =ps.id 
LEFT JOIN prison_history ph ON ph.person_id =ps.id
LEFT JOIN charge_data chd ON chd.person_id = ps.id 
WHERE DATE(ps.compas_screening_date) <= '2014-04-01'"""

In [49]:
import sqlite3

con = sqlite3.connect("../../data-ProPublica/compas.db")
df_query = pd.read_sql_query(query, con)

In [50]:
df_query["days_b_screening_arrest"] = pd.to_numeric(df_query["days_b_screening_arrest"])

df_query = df_query[
        (df_query["days_b_screening_arrest"]<=30) 
        & (df_query["days_b_screening_arrest"]>=-30) 
        & (df_query["is_recid"]!=-1) 
        & ((df_query["c_charge_degree_detailed"]!="O")
        & (df_query["c_charge_degree_detailed"]!="(CO3)")
        & (df_query["c_charge_degree_detailed"]!="(NI0)")
        & (df_query["c_charge_degree_detailed"]!="(X)")) 
      ].dropna(subset=['score_text'])

In [51]:
df_query_sub = df_query[['id', 'prior_cases_count', 'total_misd_charges','total_felon_charges', 'most_sev_misd', 'max_sev_fel', 'min_sev_fel', 'times_in_jail', 'total_days_in_jail', 'times_in_prison', 'total_days_in_prison', 'total_ordoffens_charges', 'c_charge_degree_detailed']].copy()

In [52]:
df_comb = pd.merge(df,df_query_sub,how="left", on="id")

In [53]:
# Select columns of interest and store as new df
columns_int = ["id", "name", "age", "age_cat", "sex", "race",  #"marital_status", #demographics
               "juv_fel_count", "juv_misd_count", "juv_other_count", "priors_count", 'prior_cases_count', 
               'total_ordoffens_charges', 'total_misd_charges','total_felon_charges', 'most_sev_misd', 'max_sev_fel', 
               'min_sev_fel', 'times_in_jail', 'total_days_in_jail', 'times_in_prison', 'total_days_in_prison',#criminal history
               "c_charge_degree", "c_charge_degree_detailed", "c_charge_desc", #current charges
               "score_text", "decile_score", #compas results for current charges -> target for regression (COMPAS scores for general recidivism)
               #"r_charge_degree", "r_charge_desc", #commited crimes (general) within 2 years after COMPAS scoring (no feature for final model, but relevant fro feature engineering)
               "two_year_recid"] #target for classification 
df_com = df_comb[columns_int].drop_duplicates().copy()

In [54]:
# Cut off parenthesis
df_com[["most_sev_misd", "max_sev_fel", "min_sev_fel", "c_charge_degree_detailed"]] = df_com[["most_sev_misd", "max_sev_fel", "min_sev_fel", "c_charge_degree_detailed"]].apply(lambda x: x.str.replace("[()]","")).fillna(0)

/var/folders/r8/ml9zphbs05z5wx9ykw9g7r_40000gn/T/ipykernel_36508/1015238502.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_com[["most_sev_misd", "max_sev_fel", "min_sev_fel", "c_charge_degree_detailed"]] = df_com[["most_sev_misd", "max_sev_fel", "min_sev_fel", "c_charge_degree_detailed"]].apply(lambda x: x.str.replace("[()]","")).fillna(0)


In [55]:
df_com.shape

(5304, 27)

In [57]:
#lambda x: x if df_com.max_sev_fel + df_com.min_sev_fel

In [58]:
df_com = df_com.set_index("id")

In [59]:
df_com.columns

Index(['name', 'age', 'age_cat', 'sex', 'race', 'juv_fel_count',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'prior_cases_count', 'total_ordoffens_charges', 'total_misd_charges',
       'total_felon_charges', 'most_sev_misd', 'max_sev_fel', 'min_sev_fel',
       'times_in_jail', 'total_days_in_jail', 'times_in_prison',
       'total_days_in_prison', 'c_charge_degree', 'c_charge_degree_detailed',
       'c_charge_desc', 'score_text', 'decile_score', 'two_year_recid'],
      dtype='object')

In [60]:
columns = ['age', 'sex', 'race', 'juv_fel_count',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'prior_cases_count', 'total_ordoffens_charges', 'total_misd_charges',
       'total_felon_charges', 'most_sev_misd', 'max_sev_fel', 'min_sev_fel',
       'times_in_jail', 'total_days_in_jail', 'times_in_prison',
       'total_days_in_prison', 'c_charge_degree_detailed','two_year_recid']

df_final = df_com[columns].copy()

In [114]:
df_final

,age,sex,race,juv_fel_count,juv_misd_count,juv_other_count,priors_count,prior_cases_count,total_ordoffens_charges,total_misd_charges,total_felon_charges,most_sev_misd,max_sev_fel,min_sev_fel,times_in_jail,total_days_in_jail,times_in_prison,total_days_in_prison,c_charge_degree_detailed,two_year_recid
id,,,,,,,,,,,,,,,,,,,,
1,69,Male,Other,0,0,0,0,0.0,4.0,0.0,0.0,0,0,0,1.0,0.984468,0.0,0.0,F3,0
3,34,Male,African-American,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,1.0,10.077384,0.0,0.0,F3,1
4,24,Male,African-American,0,0,1,4,4.0,6.0,7.0,3.0,M1,F3,F3,1.0,1.085764,0.0,0.0,F3,1
7,44,Male,Other,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.000000,0.0,0.0,M1,0
8,41,Male,Caucasian,0,0,0,14,14.0,4.0,6.0,15.0,M1,F3,F2,1.0,6.298681,2.0,1065.0,F3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10995,20,Male,African-American,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,1.0,0.872118,0.0,0.0,F3,0
10996,23,Male,African-American,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,1.0,1.903391,0.0,0.0,F3,0
10997,23,Male,African-American,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,1.0,1.868032,0.0,0.0,F3,0


In [61]:
X = df_final.drop(columns="two_year_recid")
y = df_final.two_year_recid

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

In [74]:
feat_ordinal_dict = {
    #"age_cat": ["Less than 25", "25 - 45", "Greater than 45"],
    #"sex": ["Male","Female"],
    #"c_charge_degree": ["M", "F"],
    "c_charge_degree_detailed": ["0","M2","M1","F3","F2","F1"],
    "most_sev_misd": ["0","MO3","M2","M1"],
    "max_sev_fel": ["0", "F3","F2","F1", "F5", "F6", "F7"],
    "min_sev_fel": ["0", "F3","F2","F1", "F5", "F6", "F7"]
#    "score_text": ["Low", "Medium", "High"]
    }

feat_ordinal = sorted(feat_ordinal_dict.keys()) # sort alphabetically
feat_ordinal_values_sorted = [feat_ordinal_dict[i] for i in feat_ordinal]

encoder_ordinal = OrdinalEncoder(
    categories=feat_ordinal_values_sorted,
    dtype= np.int64,
    handle_unknown="use_encoded_value",
    unknown_value=-1 # Considers unknown values as worse than missing
)

preproc_ordinal = make_pipeline(
    SimpleImputer(strategy="constant", fill_value=0),
    encoder_ordinal,
    #KNNImputer(missing_values=-1, n_neighbors=25), #kNN impute missing values
#    StandardScaler()
    MinMaxScaler()
)

preproc_ordinal

Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value=0, strategy='constant')),
                ('ordinalencoder',
                 OrdinalEncoder(categories=[['0', 'M2', 'M1', 'F3', 'F2', 'F1'],
                                            ['0', 'F3', 'F2', 'F1', 'F5', 'F6',
                                             'F7'],
                                            ['0', 'F3', 'F2', 'F1', 'F5', 'F6',
                                             'F7'],
                                            ['0', 'MO3', 'M2', 'M1']],
                                dtype=<class 'numpy.int64'>,
                                handle_unknown='use_encoded_value',
                                unknown_value=-1)),
                ('minmaxscaler', MinMaxScaler())])

In [75]:
# Define numerical feature once-for-all
#from sklearn.compose import make_column_selector
#feat_numerical = make_column_selector(dtype_include='number')

feat_numerical = sorted(X.select_dtypes(include=["int64", "float64"]).columns)
#feat_numerical = sorted(df_clas.select_dtypes(include=["int64", "float64"]).columns)


preproc_numerical = make_pipeline(
    SimpleImputer(),
#    StandardScaler(),
    MinMaxScaler()
)

In [78]:
# Define nominal features to one-hot-encode as the remaining ones (non numerical, non ordinal)
feat_nominal = sorted(list(set(X.columns) - set(feat_numerical) - set(feat_ordinal)))

preproc_nominal = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore", drop="if_binary")
)

In [80]:
preproc = make_column_transformer(
    (preproc_numerical, feat_numerical),
    (preproc_ordinal, feat_ordinal),
    (preproc_nominal, feat_nominal), 
    remainder="drop")

preproc

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('minmaxscaler',
                                                  MinMaxScaler())]),
                                 ['age', 'juv_fel_count', 'juv_misd_count',
                                  'juv_other_count', 'prior_cases_count',
                                  'priors_count', 'times_in_jail',
                                  'times_in_prison', 'total_days_in_jail',
                                  'total_days_in_prison', 'total_felon_charges',
                                  'total_misd_charges',
                                  'total...
                                                                 dtype=<class 'numpy.int64'>,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1)),
                                                 ('minmaxscaler',
                                                  MinMaxScaler())]),
                                 ['c_charge_degree_detailed', 'max_sev_fel',
                                  'min_sev_fel', 'most_sev_misd']),
                                ('pipeline-3',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'))]),
                                 ['race', 'sex'])])

In [81]:
pipe_log = make_pipeline(preproc, LogisticRegression())
pipe_log

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  ['age', 'juv_fel_count',
                                                   'juv_misd_count',
                                                   'juv_other_count',
                                                   'prior_cases_count',
                                                   'priors_count',
                                                   'times_in_jail',
                                                   'times_in_prison',
                                                   'total_days_in_jail',
                                                   'total_days_in_prison',
                                                   'total_felon...
                                                                                  unknown_value=-1)),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  ['c_charge_degree_detailed',
                                                   'max_sev_fel', 'min_sev_fel',
                                                   'most_sev_misd']),
                                                 ('pipeline-3',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['race', 'sex'])])),
                ('logisticregression', LogisticRegression())])

In [91]:
cv_res = cross_validate(pipe_log, X_train, y_train, scoring="roc_auc", cv=10)

In [92]:
cv_res["test_score"].mean()

0.743134344258399

In [94]:
pipe_log.fit(X_train, y_train)
y_pred = pipe_log.predict(X_test)

roc_auc_score(y_test, y_pred)

0.65061662380431

In [87]:
pipe_forest = make_pipeline(preproc, RandomForestClassifier())

In [93]:
cv_res = cross_validate(pipe_forest, X_train, y_train, scoring="roc_auc", cv=10)

In [89]:
cv_res["test_score"].mean()

0.7205630745235811

In [95]:
pipe_forest.fit(X_train, y_train)
y_pred = pipe_forest.predict(X_test)

roc_auc_score(y_test, y_pred)

0.647402371434149

In [100]:
confusion_matrix(y_test, y_pred)

array([[815, 192],
       [301, 284]])

In [99]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.81      0.77      1007
           1       0.60      0.49      0.54       585

    accuracy                           0.69      1592
   macro avg       0.66      0.65      0.65      1592
weighted avg       0.68      0.69      0.68      1592



In [104]:
pipe_tree = make_pipeline(preproc, DecisionTreeClassifier())

In [105]:
pipe_tree.fit(X_train, y_train)
y_pred = pipe_tree.predict(X_test)

roc_auc_score(y_test, y_pred)

0.5892521579711252